In [1]:
!git clone https://github.com/BenchCouncil/AIGCBench.git
%cd AIGCBench

Cloning into 'AIGCBench'...
remote: Enumerating objects: 65, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 65 (delta 29), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (65/65), 6.52 MiB | 12.76 MiB/s, done.
Resolving deltas: 100% (29/29), done.
/content/AIGCBench


In [ ]:
!pip install torch torchvision torchaudio
!pip install transformers
!pip install opencv-python-headless
!pip install scikit-image

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [ ]:
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git
!pip install einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 2.2 MB/s eta 0:00:00
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-84qm45c1
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-84qm45c1
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369490 sha256=63ef75a574a55990464a1d413b85e00d53711379f0a6182c9b010f2b847e8075
  Stored in directory: /tmp/pip-ephem-wheel-cache-saxao5q_/wheels/da/2b/4c/d6691fa9597aac8bb85d2ac13b112deb897d5b50f5ad9a37e4
Successfully built clip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.4 MB/s eta 0:00:00


In [ ]:
!mkdir -p models
!wget -O models/ViT-L-14.pt https://openaipublic.azureedge.net/clip/models/b8cca3fd41ae0c99ba7e8951adf17d267cdb84cd88be6f7c2e0eca1737a03836/ViT-L-14.pt

--2024-07-15 07:33:52--  https://openaipublic.azureedge.net/clip/models/b8cca3fd41ae0c99ba7e8951adf17d267cdb84cd88be6f7c2e0eca1737a03836/ViT-L-14.pt
Resolving openaipublic.azureedge.net (openaipublic.azureedge.net)... 13.107.246.59, 2620:1ec:bdf::59
Connecting to openaipublic.azureedge.net (openaipublic.azureedge.net)|13.107.246.59|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 932768134 (890M) [application/octet-stream]
Saving to: ‘models/ViT-L-14.pt’

models/ViT-L-14.pt  100%[===================>] 889.56M   141MB/s    in 6.6s    

2024-07-15 07:33:59 (135 MB/s) - ‘models/ViT-L-14.pt’ saved [932768134/932768134]



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!sed -i 's|path_to_dir|/content/AIGCBench/models|g' eval.py

### The below cell is updating the compute_video_video_similarity method for smaller frames

In [ ]:
%%writefile /content/AIGCBench/eval.py

import clip
from PIL import Image
import numpy as np
from einops import rearrange
from metrics.clip_score import calculate_clip_score
from utils import load_video, load_image
from skimage.metrics import mean_squared_error, structural_similarity as ssim

model, preprocess = clip.load("/content/AIGCBench/models/ViT-L-14.pt", device="cuda")

# GenVideo-RefVideo SSIM
def compute_video_video_similarity_ssim(reference_video_path,
                                        video_path,
                                        keyframes=[0, 4, 8, 12, 15]):
    global model, preprocess

    reference_video = load_video(reference_video_path)
    try:
        video_frames = load_video(video_path,
                                  size=(reference_video[0].shape[1],
                                        reference_video[0].shape[0]))[:16]
    except:
        print(video_path, ' Error !!')
        return {'ssim': 0, 'state': 0}

    all_frame_scores = 0.
    for idx, frames in enumerate(keyframes):
        im1 = np.array(
            reference_video[10 + frames * 3]
        )  # In webvid, we start with the tenth frame of the video for the motion. Different fps * 3
        im2 = np.array(video_frames[frames])

        ssim_value = ssim(im1,
                          im2,
                          multichannel=True,
                          win_size=7,
                          channel_axis=-1)

        all_frame_scores += ssim_value

    all_frame_scores /= len(keyframes)
    return {'ssim': all_frame_scores, 'state': 1}

# GenVideo clip
def compute_temporal_consistency(video_path):
    global model, preprocess

    all_frame_scores = 0.
    video_frames = load_video(video_path)[:16]  # get pre 16 frames
    for i in range(1, len(video_frames)):
        pre_frame = Image.fromarray(video_frames[i - 1])
        next_frame = Image.fromarray(video_frames[i])
        score = calculate_clip_score(model,
                                     preprocess,
                                     first_data=pre_frame,
                                     second_data=next_frame,
                                     first_flag='img',
                                     second_flag='img').cpu().numpy()
        all_frame_scores += score
    return all_frame_scores / (len(video_frames) - 1)

# GenVideo-Text clip
def compute_video_text_alignment(video_path, text):
    global model, preprocess

    all_frame_scores = 0.
    video_frames = load_video(video_path, )[:16]
    assert len(video_frames) != 0, f'check video_path {video_path}!'
    for frame in video_frames:
        frame = Image.fromarray(frame)
        score = calculate_clip_score(model,
                                     preprocess,
                                     first_data=text,
                                     second_data=frame,
                                     first_flag='txt',
                                     second_flag='img').cpu().numpy()
        all_frame_scores += score
    return all_frame_scores / len(video_frames)

# GenVideo-RefVideo clip (keyframes)
def compute_video_video_similarity(reference_video_path,
                                   video_path,
                                   keyframes=[0, 4, 8, 12, 15]):
    global model, preprocess
    print("Hitting this method")
    print(f"Processing reference video: {reference_video_path}")
    reference_video = load_video(reference_video_path)
    print(f"Reference video frames: {len(reference_video)}")

    try:
        print(f"Processing video: {video_path}")
        video_frames = load_video(video_path,
                                  size=reference_video[0].shape[:2])
        print(f"Video frames: {len(video_frames)}")
    except Exception as e:
        print(f"Error loading video {video_path}: {str(e)}")
        return {'clip': 0, 'state': 0}

    all_frame_scores = 0.
    for idx, frames in enumerate(keyframes):
        try:
            print(f"Processing keyframe {frames}")
            im1 = Image.fromarray(np.array(reference_video[frames]))
            im2 = Image.fromarray(np.array(video_frames[frames]))
            score = calculate_clip_score(model,
                                         preprocess,
                                         first_data=im1,
                                         second_data=im2,
                                         first_flag='img',
                                         second_flag='img').cpu().numpy()
            all_frame_scores += score
            print(score)
        except IndexError as e:
            print(f"IndexError at keyframe {frames}: {str(e)}")
            continue
        except Exception as e:
            print(f"Error processing keyframe {frames}: {str(e)}")
            continue

    if len(keyframes) > 0:
        all_frame_scores /= len(keyframes)
    else:
        print("No keyframes were successfully processed")

    return {'clip': all_frame_scores, 'state': 1}

# MSE (First) and SSIM (First)
def compute_image_image_similarity(init_image_path, video_path):
    global model, preprocess

    # all_frame_scores = 0.
    init_image = Image.fromarray(load_image(init_image_path))
    try:
        video_frames = load_video(video_path,
                                  size=init_image.size)  # size=(512, 904)
    except:
        print(video_path, ' Error !!')
        return {'MSE': 0, 'SSIM': 0, 'state': 0}
    init_image_np = np.array(init_image)
    video_frame_np = np.array(video_frames[0])  # or 10th frames

    if init_image_np.shape == video_frame_np.shape:
        mse_value = mean_squared_error(init_image_np, video_frame_np)
        ssim_value = ssim(init_image_np,
                          video_frame_np,
                          multichannel=True,
                          win_size=7,
                          channel_axis=-1)
    else:
        print("Error: The images do not have the same dimensions.", video_path)
        return {'MSE': 0, 'SSIM': 0, 'state': 0}

    return {'MSE': mse_value, 'SSIM': ssim_value, 'state': 1}

# Image-GenVideo clip
def compute_video_image_similarity(video_path, image_path):
    global model, preprocess

    all_frame_scores = 0.
    image = Image.fromarray(load_image(image_path))

    try:
        video_frames = load_video(video_path, size=image.size)
    except:
        print(video_path, ' Error !!')
        return {"clip_per": all_frame_scores / len(video_frames), "state": 0}

    for frame in video_frames:
        frame = Image.fromarray(frame)
        score = calculate_clip_score(model,
                                     preprocess,
                                     first_data=image,
                                     second_data=frame,
                                     first_flag='img',
                                     second_flag='img').cpu().numpy()
        all_frame_scores += score
    return {"clip_per": all_frame_scores / len(video_frames), "state": 1}

Overwriting /content/AIGCBench/eval.py


### Got the videos from the google drive

Google drive url is https://drive.google.com/drive/folders/1bpW3URL4S8kFG0vSlFSJ1zGEsidIvsjL?usp=drive_link

In [ ]:
import os
import glob

# Path to your videos in Google Drive
video_folder = '/content/drive/MyDrive/Q-Align_data'

# Get a list of all video files (assuming they're mp4, but adjust if needed)
video_path_list = sorted(glob.glob(os.path.join(video_folder, '*.mp4')))

print(f"Found {len(video_path_list)} videos.")

Found 30 videos.


In [ ]:
video_path_list[20]

'/content/drive/MyDrive/Q-Align_data/Waterfalling_Decent.mp4'

In [ ]:
# Use the first video as reference
from eval import compute_video_video_similarity
ref_video_path = video_path_list[0]

sum_res = 0
cnt = 0

for video_path in video_path_list[1:]:  # Skip the reference video
    res = compute_video_video_similarity(ref_video_path, video_path)
    sum_res += res['clip']
    cnt += res["state"]

if cnt > 0:
    print(f"Average CLIP similarity: {sum_res / cnt}")
else:
    print("No videos were processed.")

Hitting this method
Processing reference video: /content/drive/MyDrive/Q-Align_data/Beach.mp4
Reference video frames: 16
Processing video: /content/drive/MyDrive/Q-Align_data/BeachwithProduct.mp4
Video frames: 16
Processing keyframe 0
0.72373533
Processing keyframe 4
0.76752216
Processing keyframe 8
0.7308178
Processing keyframe 12
0.69993424
Processing keyframe 15
0.7428811
Hitting this method
Processing reference video: /content/drive/MyDrive/Q-Align_data/Beach.mp4
Reference video frames: 16
Processing video: /content/drive/MyDrive/Q-Align_data/ExtraHand.mp4
Video frames: 16
Processing keyframe 0
0.6805366
Processing keyframe 4
0.7240991
Processing keyframe 8
0.68420404
Processing keyframe 12
0.6692717
Processing keyframe 15
0.5334763
Hitting this method
Processing reference video: /content/drive/MyDrive/Q-Align_data/Beach.mp4
Reference video frames: 16
Processing video: /content/drive/MyDrive/Q-Align_data/EyesDistorted.mp4
Video frames: 16
Processing keyframe 0
0.66286874
Processing

In [ ]:
from eval import compute_video_text_alignment
compute_video_text_alignment(video_path_list[20],"a dog playing with a ball")

0.0777246505022049

In [ ]:
from eval import compute_temporal_consistency
ref_video_path='/content/drive/MyDrive/Bad videos/object4.gif'
compute_temporal_consistency(ref_video_path)

0.8395310878753662

In [ ]:
compute_temporal_consistency(video_path_list[4])

0.9576089779535929

In [ ]:
for video_path in video_path_list:  # Skip the reference video
    res = compute_temporal_consistency( video_path)

    print(res)
    print(video_path)

0.9812637448310852
/content/drive/MyDrive/Q-Align_data/Beach.mp4
0.9917521993319194
/content/drive/MyDrive/Q-Align_data/BeachwithProduct.mp4
0.9488892356554667
/content/drive/MyDrive/Q-Align_data/ExtraHand.mp4
0.9784121751785279
/content/drive/MyDrive/Q-Align_data/EyesDistorted.mp4
0.9612833460172018
/content/drive/MyDrive/Q-Align_data/FLowers_anatomy.mp4
0.9576089779535929
/content/drive/MyDrive/Q-Align_data/Face_Bad.mp4
0.9963203390439351
/content/drive/MyDrive/Q-Align_data/HeadAndShoulder.mp4
0.9350598951180776
/content/drive/MyDrive/Q-Align_data/Light1.mp4
0.9864495992660522
/content/drive/MyDrive/Q-Align_data/Plum.mp4
0.9350598951180776
/content/drive/MyDrive/Q-Align_data/ShampooBottle.mp4
0.9962668259938557
/content/drive/MyDrive/Q-Align_data/Smile.mp4
0.9983378330866496
/content/drive/MyDrive/Q-Align_data/SmillingFace.mp4
0.9703799525896708
/content/drive/MyDrive/Q-Align_data/SpiderMan_Bad.mp4
0.9907313903172811
/content/drive/MyDrive/Q-Align_data/Sunrise.mp4
0.9887410481770833


In [ ]:
for video_path in video_path_list:  # Skip the reference video
    res = compute_temporal_consistency( video_path)

    print(res)
    print(video_path)

0.960559896628062
/content/drive/MyDrive/Q-Align_data/Anatomy_Bad.mp4
0.9812637448310852
/content/drive/MyDrive/Q-Align_data/Beach_Good.mp4
0.9917521993319194
/content/drive/MyDrive/Q-Align_data/BeachwithProduct_Good.mp4
0.9286013563474019
/content/drive/MyDrive/Q-Align_data/Car_Moving_Backlwards.mp4
0.9488892356554667
/content/drive/MyDrive/Q-Align_data/ExtraHand_Bad.mp4
0.9784121751785279
/content/drive/MyDrive/Q-Align_data/EyesDistorted_Bad.mp4
0.9612833460172018
/content/drive/MyDrive/Q-Align_data/FLowers_anatomy.mp4
0.9576089779535929
/content/drive/MyDrive/Q-Align_data/Face_Bad.mp4
0.9598904848098755
/content/drive/MyDrive/Q-Align_data/Girl_Good.mp4
0.9963203390439351
/content/drive/MyDrive/Q-Align_data/HeadAndShoulder_Good.mp4
0.9655776103337605
/content/drive/MyDrive/Q-Align_data/Human_Moving_Backward.mp4
0.9350598951180776
/content/drive/MyDrive/Q-Align_data/Light1_Bad.mp4
0.9980235616366069
/content/drive/MyDrive/Q-Align_data/Mask_good.mp4
0.9864495992660522
/content/drive/My

For the bad ones, the result is somewhere around 0.96 - 0.97. But some images where the limbs are not distorted or we don't see much of deformity in humans, the score is good, even if the image is bad. This is proabably because the human images are preserved.